In [37]:
import json
import pandas as pd
import numpy as np

In [6]:
with open('../../dataset/train.json') as f:
    data= json.load(f)


In [9]:
data=pandas.DataFrame(data)

In [10]:
data.head()

,audio_embedding,is_turkey,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",0,kDCk3hLIVXo,70,60
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",1,DPcGzqHoo7Y,40,30
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",1,7yM63MTHh5k,240,230
3,"[[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...",1,luG3RmUAxxM,520,510
4,"[[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...",0,PIm3cjxTpOk,10,0


In [12]:
data['period']=data['end_time_seconds_youtube_clip']-data['start_time_seconds_youtube_clip']

In [41]:
data[data['period']!=data['audio_embedding'].str.len()]

,audio_embedding,is_turkey,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip,period
35,"[[168, 22, 202, 146, 204, 82, 85, 42, 179, 141...",0,Lv8wwTjOj1A,28,18,10
47,"[[189, 37, 224, 112, 192, 101, 114, 40, 156, 1...",0,DGGmJUd8a70,31,21,10
76,"[[173, 13, 164, 164, 162, 16, 154, 22, 195, 71...",0,ZCXTEgvXJos,30,20,10
97,"[[175, 20, 147, 141, 199, 70, 89, 47, 142, 200...",0,DPXqBXhAr7M,3,0,3
120,"[[151, 18, 150, 56, 240, 86, 108, 101, 134, 19...",1,JBj7Sr9A-m8,25,15,10
...,...,...,...,...,...,...
1121,"[[154, 18, 175, 78, 194, 59, 63, 45, 175, 226,...",1,f2AsH6WwBXA,33,23,10
1124,"[[174, 22, 160, 119, 199, 0, 135, 126, 132, 12...",1,0OGaY_4Ns6s,35,25,10
1126,"[[172, 44, 194, 142, 255, 47, 76, 113, 196, 16...",0,yB6V_pv9j5Q,70,60,10
1133,"[[166, 22, 198, 108, 184, 59, 77, 82, 196, 183...",0,XZnT6aLbe34,7,0,7


In [60]:
with open('../../dataset/test.json') as f:
    test_data= json.load(f)

In [63]:
test_data=pd.DataFrame(test_data)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   audio_embedding                  1196 non-null   object
 1   vid_id                           1196 non-null   object
 2   end_time_seconds_youtube_clip    1196 non-null   int64 
 3   start_time_seconds_youtube_clip  1196 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 37.5+ KB


In [74]:
def mean_audio_embedding(embeddings_list):
        embeddings_array = np.array(embeddings_list)
        return np.mean(embeddings_array, axis=0)
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

def train_and_evaluate_lr(data):
    """
    Train and evaluate multiple classifiers on the mean of VGGish embeddings for each audio segment using GridSearchCV.

    Args:
        data: DataFrame with 'audio_embedding' (list of embeddings) and 'is_turkey'.

    Returns:
        best_estimators: Dictionary of best estimators for each model.
    """

    X = np.stack([mean_audio_embedding(emb) for emb in tqdm(data['audio_embedding'], desc="Extracting mean embeddings")])
    y = data['is_turkey']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    models = {
        "LogisticRegression": (LogisticRegression(random_state=42), {
            'C': [0.001, 0.01, 0.1, 1, 10],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga'],
            'max_iter': [1000]
        }),
        "RandomForest": (RandomForestClassifier(random_state=42), {
            'n_estimators': [50, 100],
            'max_depth': [5, 10, None]
        }),
        "GradientBoosting": (GradientBoostingClassifier(random_state=42), {
            'n_estimators': [50, 100],
            'learning_rate': [0.01, 0.1, 1.0]
        }),
        "SVC": (SVC(random_state=42), {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        })
    }

    best_estimators = {}
    for name, (model, param_grid) in models.items():
        print(f"\nTraining {name}...")
        grid = GridSearchCV(model, param_grid, cv=3, n_jobs=-1)
        grid.fit(X_train_scaled, y_train)
        print(f"Best parameters for {name}:", grid.best_params_)
        y_pred = grid.predict(X_test_scaled)
        print(f"Accuracy for {name}:", accuracy_score(y_test, y_pred))
        print(f"Classification Report for {name}:\n", classification_report(y_test, y_pred))
        best_estimators[name] = grid.best_estimator_

    return best_estimators


In [69]:
# Compute mean audio embeddings for test_data
X_test = np.stack([mean_audio_embedding(emb) for emb in tqdm(test_data['audio_embedding'], desc="Extracting mean embeddings")])

# Predict using the trained model
model = train_and_evaluate_lr(data)
y_pred_test = model.predict(X_test)
test_data['is_turkey'] = y_pred_test.astype(int)
submission = test_data[['vid_id', 'is_turkey']]
submission.to_csv('submission.csv', index=False)


Extracting mean embeddings: 100%|██████████| 1195/1195 [00:00<00:00, 6294.65it/s]


Best parameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
Accuracy: 0.9414225941422594
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.97      0.95       141
           1       0.96      0.90      0.93        98

    accuracy                           0.94       239
   macro avg       0.94      0.93      0.94       239
weighted avg       0.94      0.94      0.94       239



In [79]:
train_and_evaluate_lr(data)

Extracting mean embeddings: 100%|██████████| 1195/1195 [00:00<00:00, 4819.17it/s]



Training LogisticRegression...
Best parameters for LogisticRegression: {'C': 0.01, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'saga'}
Accuracy for LogisticRegression: 0.9414225941422594
Classification Report for LogisticRegression:
               precision    recall  f1-score   support

           0       0.93      0.97      0.95       141
           1       0.96      0.90      0.93        98

    accuracy                           0.94       239
   macro avg       0.94      0.93      0.94       239
weighted avg       0.94      0.94      0.94       239


Training RandomForest...
Best parameters for RandomForest: {'max_depth': None, 'n_estimators': 100}
Accuracy for RandomForest: 0.9246861924686193
Classification Report for RandomForest:
               precision    recall  f1-score   support

           0       0.91      0.96      0.94       141
           1       0.94      0.87      0.90        98

    accuracy                           0.92       239
   macro avg       0.93      0.

{'LogisticRegression': LogisticRegression(C=0.01, max_iter=1000, random_state=42, solver='saga'),
 'RandomForest': RandomForestClassifier(random_state=42),
 'GradientBoosting': GradientBoostingClassifier(learning_rate=1.0, random_state=42),
 'SVC': SVC(C=10, random_state=42)}